In [1]:
import os
import re
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import datetime as dt
import random
import json
import requests
import undetected_chromedriver as uc


# REQUIRED FUNCTIONS

def get_file_data(file_name):
    with open(file=file_name, mode="r", encoding="utf-8") as file:
        data = file.read().strip().split("\n")
    return data


def find_element_send_text(ele, text, clear=True):
    while True:
        try:
            input_field = driver.find_element(By.XPATH, ele)
            if clear:
                input_field.clear()
            input_field.send_keys(text)

            break
        except:
            time.sleep(0.1)


def specific_clicker(ele):
    while True:
        try:
            element = driver.find_element(By.XPATH, ele)
            webdriver.ActionChains(driver).move_to_element_with_offset(element, 1, 0).click(element).perform()

            break
        except Exception as e:
            print(e)
            pass


def specific_clicker2(ele):
    try:
        element = driver.find_element(By.XPATH, ele)
        webdriver.ActionChains(driver).move_to_element_with_offset(element, 1, 0).click(element).perform()

    except Exception as e:
        print(e)
        pass


def drag_and_drop_file(drop_target, path):
    """
    Drag and drop the provided file path onto the provided element target.


    Parameters
    ----------
    drop_target : WebElement
        The web element to drop the file at path on

    path : str
        The file path to drag onto the web element

    Returns
    -------
    bool
        Whether or not dragging the file was successful

    """

    # https://stackoverflow.com/questions/43382447/python-with-selenium-drag-and-drop-from-file-system-to-webdriver
    JS_DROP_FILE = """
            var target = arguments[0],
                offsetX = arguments[1],
                offsetY = arguments[2],
                document = target.ownerDocument || document,
                window = document.defaultView || window;

            var input = document.createElement('INPUT');
            input.type = 'file';
            input.onchange = function () {
              var rect = target.getBoundingClientRect(),
                  x = rect.left + (offsetX || (rect.width >> 1)),
                  y = rect.top + (offsetY || (rect.height >> 1)),
                  dataTransfer = { files: this.files };

              ['dragenter', 'dragover', 'drop'].forEach(function (name) {
                var evt = document.createEvent('MouseEvent');
                evt.initMouseEvent(name, !0, !0, window, 0, 0, 0, x, y, !1, !1, !1, !1, 0, null);
                evt.dataTransfer = dataTransfer;
                target.dispatchEvent(evt);
              });

              setTimeout(function () { document.body.removeChild(input); }, 25);
            };
            document.body.appendChild(input);
            return input;
        """
    try:

        # BUG: requires double to register file upload
        file_input = drop_target.parent.execute_script(JS_DROP_FILE, drop_target, 0, 0)
        file_input.send_keys(path)
        file_input = drop_target.parent.execute_script(JS_DROP_FILE, drop_target, 50, 50)
        file_input.send_keys(path)

        return True
    except Exception as e:
        print(e)

    return False


def close_chrome_app():
    try:
        os.system("taskkill /f /im chrome.exe")
    except:
        pass


In [2]:
close_chrome_app()
options = uc.ChromeOptions()
path = os.path.join(os.getenv('LOCALAPPDATA'), 'Google', 'Chrome', 'User Data')
options.add_argument(fr"user-data-dir={path}")
options.add_argument(f'--profile-directory=Profile 3')
driver = uc.Chrome(options=options)
driver.maximize_window()

In [3]:
driver.get("https://onlyfans.com/my/queue")

In [5]:
driver.find_element(By.XPATH, '(//input[@class="vdatetime-input form-control g-input"])[1]').send_keys('5/16/2023')

In [ ]:
driver.get("https://twitter.com/")

In [8]:
time_post = '14:07'.split(':')
if int(time_post[0]) > 12:
    time_post[0] = str(int(time_post[0]) - 12)
    time_post[1] = time_post[1] + ' PM'
else:
    time_post[1] = time_post[1] + ' AM'

time_post = ':'.join(time_post)

In [9]:
print(time_post)

2:07 PM


In [21]:
with open('schedules.json') as f:
    config = json.load(f)['schedule_data']
    data = [schedule for schedule in config if schedule['platform'] == 'twitter']

print("Starting Chrome")
close_chrome_app()
options = uc.ChromeOptions()
path = os.path.join(os.getenv('LOCALAPPDATA'), 'Google', 'Chrome', 'User Data')
options.add_argument(fr"user-data-dir={path}")
options.add_argument(f'--profile-directory=Default')
driver = uc.Chrome(options=options)
driver.maximize_window()


Starting Chrome


In [22]:
import pyperclip
from selenium.webdriver.common.keys import Keys

for index, single_post in enumerate(data):
    driver.get('https://twitter.com/home')
    specific_clicker('//*[@aria-label="Schedule Tweet"]')
    for i in range(3):
        specific_clicker2('//*[@aria-label="Schedule Tweet"]')

    time.sleep(2)

    # SET POST MONTH: 2023-04-12 -> April
    month_post = data[index]['post_date'].split('-')[1]
    month_post = str(datetime.strptime(month_post, '%m').strftime('%B')).title()
    driver.find_element(By.XPATH, f'//*[text()="{month_post}"]').click()

    # SET POST DAY: 2023-04-12 -> 12
    day_post = data[index]['post_date'].split('-')[2]
    driver.find_element(By.XPATH, f'//*[@id="SELECTOR_2"]//*[text()="{day_post}"]').click()

    # SET POST YEAR: 2023-04-12 -> 2023
    year_post = data[index]['post_date'].split('-')[0]
    driver.find_element(By.XPATH, f'//*[@id="SELECTOR_3"]//*[text()="{year_post}"]').click()

    time_post = data[index]['post_time']
    hour_post = time_post.split(':')[0]
    minute_post = str(time_post.split(':')[1].split(' ')[0])

    # SET POST HOUR
    driver.find_element(By.XPATH, f'//*[@id="SELECTOR_4"]//*[text()="{hour_post}"]').click()

    # SET POST MINUTE
    driver.find_element(By.XPATH, f'//*[@id="SELECTOR_5"]//*[text()="{minute_post}"]').click()

    specific_clicker("//*[text()='Confirm']")

    while True:
        try:
            driver.find_element(By.XPATH, '//input[@data-testid="fileInput"]').send_keys(
                r'C:\Users\shahb\PycharmProjects\ClientsData\SIA PROJECT\download_files\Quiz-Website.png')
            break
        except:
            pass

    pyperclip.copy('hello world')
    driver.find_element(By.XPATH, '//div[@aria-label="Tweet text"]').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)
    specific_clicker("//*[text()='Schedule']")
    time.sleep(4)#%%
import os
import re
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import datetime as dt
import random
import json
import requests
import undetected_chromedriver as uc


# REQUIRED FUNCTIONS

def get_file_data(file_name):
    with open(file=file_name, mode="r", encoding="utf-8") as file:
        data = file.read().strip().split("\n")
    return data


def find_element_send_text(ele, text, clear=True):
    while True:
        try:
            input_field = driver.find_element(By.XPATH, ele)
            if clear:
                input_field.clear()
            input_field.send_keys(text)

            break
        except:
            time.sleep(0.1)


def specific_clicker(ele):
    while True:
        try:
            element = driver.find_element(By.XPATH, ele)
            webdriver.ActionChains(driver).move_to_element_with_offset(element, 1, 0).click(element).perform()

            break
        except Exception as e:
            print(e)
            pass


def specific_clicker2(ele):
    try:
        element = driver.find_element(By.XPATH, ele)
        webdriver.ActionChains(driver).move_to_element_with_offset(element, 1, 0).click(element).perform()

    except Exception as e:
        print(e)
        pass


def drag_and_drop_file(drop_target, path):
    """
    Drag and drop the provided file path onto the provided element target.


    Parameters
    ----------
    drop_target : WebElement
        The web element to drop the file at path on

    path : str
        The file path to drag onto the web element

    Returns
    -------
    bool
        Whether or not dragging the file was successful

    """

    # https://stackoverflow.com/questions/43382447/python-with-selenium-drag-and-drop-from-file-system-to-webdriver
    JS_DROP_FILE = """
            var target = arguments[0],
                offsetX = arguments[1],
                offsetY = arguments[2],
                document = target.ownerDocument || document,
                window = document.defaultView || window;

            var input = document.createElement('INPUT');
            input.type = 'file';
            input.onchange = function () {
              var rect = target.getBoundingClientRect(),
                  x = rect.left + (offsetX || (rect.width >> 1)),
                  y = rect.top + (offsetY || (rect.height >> 1)),
                  dataTransfer = { files: this.files };

              ['dragenter', 'dragover', 'drop'].forEach(function (name) {
                var evt = document.createEvent('MouseEvent');
                evt.initMouseEvent(name, !0, !0, window, 0, 0, 0, x, y, !1, !1, !1, !1, 0, null);
                evt.dataTransfer = dataTransfer;
                target.dispatchEvent(evt);
              });

              setTimeout(function () { document.body.removeChild(input); }, 25);
            };
            document.body.appendChild(input);
            return input;
        """
    try:

        # BUG: requires double to register file upload
        file_input = drop_target.parent.execute_script(JS_DROP_FILE, drop_target, 0, 0)
        file_input.send_keys(path)
        file_input = drop_target.parent.execute_script(JS_DROP_FILE, drop_target, 50, 50)
        file_input.send_keys(path)

        return True
    except Exception as e:
        print(e)

    return False


def close_chrome_app():
    try:
        os.system("taskkill /f /im chrome.exe")
    except:
        pass


Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@aria-label="Schedule Tweet"]"}
  (Session info: chrome=111.0.5563.147)
Stacktrace:
Backtrace:
	(No symbol) [0x0021DCE3]
	(No symbol) [0x001B39D1]
	(No symbol) [0x000C4DA8]
	(No symbol) [0x000F019F]
	(No symbol) [0x000F03AB]
	(No symbol) [0x0011EE62]
	(No symbol) [0x0010AF14]
	(No symbol) [0x0011D57C]
	(No symbol) [0x0010ACC6]
	(No symbol) [0x000E6F68]
	(No symbol) [0x000E80CD]
	GetHandleVerifier [0x00493832+2506274]
	GetHandleVerifier [0x004C9794+2727300]
	GetHandleVerifier [0x004CE36C+2746716]
	GetHandleVerifier [0x002C6690+617600]
	(No symbol) [0x001BC712]
	(No symbol) [0x001C1FF8]
	(No symbol) [0x001C20DB]
	(No symbol) [0x001CC63B]
	BaseThreadInitThunk [0x754E00F9+25]
	RtlGetAppContainerNamedObjectPath [0x76F37BBE+286]
	RtlGetAppContainerNamedObjectPath [0x76F37B8E+238]
	(No symbol) [0x00000000]

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@aria-label="Sch

In [23]:
close_chrome_app()
options = uc.ChromeOptions()
path = os.path.join(os.getenv('LOCALAPPDATA'), 'Google', 'Chrome', 'User Data')
options.add_argument(fr"user-data-dir={path}")
options.add_argument(f'--profile-directory=Profile 1')
options.add_argument("--disable-extensions")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-gpu")
options.add_argument("--disable-infobars")
options.add_argument("--disable-notifications")
driver = uc.Chrome(options=options)
driver.maximize_window()

In [ ]:
driver.get("https://twitter.com/")

In [ ]:
time_post = '14:07'.split(':')
if int(time_post[0]) > 12:
    time_post[0] = str(int(time_post[0]) - 12)
    time_post[1] = time_post[1] + ' PM'
else:
    time_post[1] = time_post[1] + ' AM'

time_post = ':'.join(time_post)

In [ ]:
print(time_post)

In [ ]:
with open('schedules.json') as f:
    config = json.load(f)['schedule_data']
    data = [schedule for schedule in config if schedule['platform'] == 'twitter']

print("Starting Chrome")
close_chrome_app()
options = uc.ChromeOptions()
path = os.path.join(os.getenv('LOCALAPPDATA'), 'Google', 'Chrome', 'User Data')
options.add_argument(fr"user-data-dir={path}")
options.add_argument(f'--profile-directory=Default')
driver = uc.Chrome(options=options)
driver.maximize_window()


In [ ]:
import pyperclip
from selenium.webdriver.common.keys import Keys

for index, single_post in enumerate(data):
    driver.get('https://twitter.com/home')
    specific_clicker('//*[@aria-label="Schedule Tweet"]')
    for i in range(3):
        specific_clicker2('//*[@aria-label="Schedule Tweet"]')

    time.sleep(2)

    # SET POST MONTH: 2023-04-12 -> April
    month_post = data[index]['post_date'].split('-')[1]
    month_post = str(datetime.strptime(month_post, '%m').strftime('%B')).title()
    driver.find_element(By.XPATH, f'//*[text()="{month_post}"]').click()

    # SET POST DAY: 2023-04-12 -> 12
    day_post = data[index]['post_date'].split('-')[2]
    driver.find_element(By.XPATH, f'//*[@id="SELECTOR_2"]//*[text()="{day_post}"]').click()

    # SET POST YEAR: 2023-04-12 -> 2023
    year_post = data[index]['post_date'].split('-')[0]
    driver.find_element(By.XPATH, f'//*[@id="SELECTOR_3"]//*[text()="{year_post}"]').click()

    time_post = data[index]['post_time']
    hour_post = time_post.split(':')[0]
    minute_post = str(time_post.split(':')[1].split(' ')[0])

    # SET POST HOUR
    driver.find_element(By.XPATH, f'//*[@id="SELECTOR_4"]//*[text()="{hour_post}"]').click()

    # SET POST MINUTE
    driver.find_element(By.XPATH, f'//*[@id="SELECTOR_5"]//*[text()="{minute_post}"]').click()

    specific_clicker("//*[text()='Confirm']")

    while True:
        try:
            driver.find_element(By.XPATH, '//input[@data-testid="fileInput"]').send_keys(
                r'C:\Users\shahb\PycharmProjects\ClientsData\SIA PROJECT\download_files\Quiz-Website.png')
            break
        except:
            pass

    pyperclip.copy('hello world')
    driver.find_element(By.XPATH, '//div[@aria-label="Tweet text"]').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)
    specific_clicker("//*[text()='Schedule']")
    time.sleep(4)


